In [ ]:
# For original code:
#   Copyright 2019 The TensorFlow Hub Authors. All Rights Reserved.
# For modifications:
#   Copyright 2022 Bucket Interior Team. All Rights Reserved.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS, 
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

In [ ]:
img_paths=["test.jpg","test2.jpg","test3.jpg"]

In [ ]:
import os
import time
from PIL import Image
import streamlit as st
import pandas as pd
import numpy as np
import tensorflow as tf
import tensorflow_hub as hub
import matplotlib.pyplot as plt
os.environ["TFHUB_DOWNLOAD_PROGRESS"] = "True"

SAVED_MODEL_PATH = "https://tfhub.dev/captain-pool/esrgan-tf2/1"

def preprocess_image(image_path):
    hr_image = tf.image.decode_image(tf.io.read_file(image_path))
      # If PNG, remove the alpha channel. The model only supports
      # images with 3 color channels.
    if hr_image.shape[-1] == 4:
        hr_image = hr_image[...,:-1]
    hr_size = (tf.convert_to_tensor(hr_image.shape[:-1]) // 4) * 4
    hr_image = tf.image.crop_to_bounding_box(hr_image, 0, 0, hr_size[0], hr_size[1])
    hr_image = tf.cast(hr_image, tf.float32)
    return tf.expand_dims(hr_image, 0)
    
    
def plot_image(image, title=""):
    image = np.asarray(image)
    image = tf.clip_by_value(image, 0, 255)
    image = Image.fromarray(tf.cast(image, tf.uint8).numpy())
    plt.imshow(image)
    plt.axis("off")
    plt.title(title)


# Defining helper functions
def downscale_image(image):
    image_size = []
    if len(image.shape) == 3:
        image_size = [image.shape[1], image.shape[0]]
    else:
        raise ValueError("Dimension mismatch. Can work only on single image.")

    image = tf.squeeze(
    tf.cast(tf.clip_by_value(image, 0, 255), tf.uint8))

    lr_image = np.asarray(Image.fromarray(image.numpy()).resize([image_size[0] // 4, image_size[1] // 4],Image.BICUBIC))

    lr_image = tf.expand_dims(lr_image, 0)
    lr_image = tf.cast(lr_image, tf.float32)
    return lr_image

model = hub.load(SAVED_MODEL_PATH)
plt.rcParams['figure.figsize'] = [25, 8 * len(img_paths)]
fig, axes = plt.subplots(len(img_paths),2)

lr_img, super_img = [],[]
for i in range(len(img_paths)):
    IMAGE_PATH=img_paths[i]
    hr_image=hr_image = preprocess_image(IMAGE_PATH)
    lr_image = downscale_image(tf.squeeze(hr_image))
    fake_image = model(lr_image)
    fake_image = tf.squeeze(fake_image)
    lr_img.append(lr_image)
    super_img.append(fake_image)
    plt.subplot(len(img_paths),2,2*i+1)
    fig.tight_layout()
    plot_image(tf.squeeze(lr_image), str(lr_image.shape[1:3]))
    plt.subplot(len(img_paths),2,2*i+2)
    fig.tight_layout()
    plot_image(tf.squeeze(fake_image), str(fake_image.shape[1:3]))

In [ ]:
super_img